# 22. QCompleter

The **QCompleter** class provides completions based on an item model.

You can use `QCompleter` to provide auto completions in any **Qt widget, such as QLineEdit and QComboBox** . When the user starts typing a word, QCompleter suggests possible ways of completing the word, based on a word list. The word list is provided as a `QAbstractItemModel` . (For simple applications, where the word list is static, you can pass a QStringList to QCompleter ‘s constructor.)

## 22.1 A simple example

In below example, we use a static list to build a QCompleter and add it to the QLineEdit. You can notice that the QCompleter has its own algo to find the match, and starts from the beginning of the string. For more detail on how it's implemented, you can visite this [page](https://doc.qt.io/qtforpython-5/PySide2/QtWidgets/QCompleter.html#more)

In [ ]:
import sys
from PyQt6.QtWidgets import (QLineEdit, QWidget, QLabel, QApplication, QVBoxLayout, QCompleter)
from PyQt6.QtCore import Qt


class MainWindow(QWidget):

    def __init__(self, *args, **kwargs):
        super().__init__()

        self.setGeometry(100, 100, 400, 300)
        self.setWindowTitle('Control Panel')

        # create a main layout
        layout = QVBoxLayout()
        self.setLayout(layout)

        # create a completer based on a list
        names = [
            "Heater", "Stove", "Living Room Light", "Balcony Light",
            "Fan", "Room Light", "Oven", "Desk Light",
            "Bedroom Heater", "Wall Switch"
        ]
        self.completer = QCompleter(names)
        # set the completer as case-insensitive
        self.completer.setCaseSensitivity(Qt.CaseSensitivity.CaseInsensitive)

        # create a QLineEdit as Search bar.
        self.searchbar = QLineEdit()
        self.searchbar.textChanged.connect(self.update_display)
        # Add Completer to search bar
        self.searchbar.setCompleter(self.completer)

        self.messageView = QLabel()

        layout.addWidget(self.searchbar)
        layout.addWidget(self.messageView)
        self.show()

    def update_display(self, text):
        self.messageView.setText(text)


app = QApplication(sys.argv)
w = MainWindow()
w.show()
sys.exit(app.exec())


## 22.2 A more complex example on QCompleter in QLineEdit

We find the auto-completion in the QCompleter does not match our needs. This time We will add something more, if the input text is a subset of anyword in the Qcompleter, we want to show it. The logic is quite simple

```python
def update_display(self, text):
        for widget in self.widgets:
            if text.lower() in widget.name.lower():
                widget.show()
            else:
                widget.hide()
```

You can replace it with more appropriate matching logic.

In [ ]:
import sys
from PyQt6.QtWidgets import (
    QWidget, QLineEdit, QLabel, QPushButton, QScrollArea, QMainWindow,
    QApplication, QHBoxLayout, QVBoxLayout, QSpacerItem, QSizePolicy, QCompleter
)
from PyQt6.QtCore import Qt
from PyQt6.QtWidgets import (QWidget, QLabel, QPushButton,
                             QHBoxLayout)


class OnOffWidget(QWidget):

    def __init__(self, name):
        super(OnOffWidget, self).__init__()

        self.name = name
        self.is_on = False

        self.lbl = QLabel(self.name)
        self.btn_on = QPushButton("On")
        self.btn_off = QPushButton("Off")

        self.hbox = QHBoxLayout()
        self.hbox.addWidget(self.lbl)
        self.hbox.addWidget(self.btn_on)
        self.hbox.addWidget(self.btn_off)

        self.btn_on.clicked.connect(self.on)
        self.btn_off.clicked.connect(self.off)

        self.setLayout(self.hbox)

        self.update_button_state()

    def show(self):
        """
        Show this widget, and all child widgets.
        """
        for w in [self, self.lbl, self.btn_on, self.btn_off]:
            w.setVisible(True)

    def hide(self):
        """
        Hide this widget, and all child widgets.
        """
        for w in [self, self.lbl, self.btn_on, self.btn_off]:
            w.setVisible(False)

    def off(self):
        self.is_on = False
        self.update_button_state()

    def on(self):
        self.is_on = True
        self.update_button_state()

    def update_button_state(self):
        """
        Update the appearance of the control buttons (On/Off)
        depending on the current state.
        """
        if self.is_on == True:
            self.btn_on.setStyleSheet("background-color: #4CAF50; color: #fff;")
            self.btn_off.setStyleSheet("background-color: none; color: none;")
        else:
            self.btn_on.setStyleSheet("background-color: none; color: none;")
            self.btn_off.setStyleSheet("background-color: #D32F2F; color: #fff;")


class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super().__init__()

        self.controls = QWidget()  # Controls container widget.
        self.controlsLayout = QVBoxLayout()  # Controls container layout.

        # List of names, widgets are stored in a dictionary by these keys.
        widget_names = [
            "Heater", "Stove", "Living Room Light", "Balcony Light",
            "Fan", "Room Light", "Oven", "Desk Light",
            "Bedroom Heater", "Wall Switch"
        ]
        self.widgets = []

        # Iterate the names, creating a new OnOffWidget for
        # each one, adding it to the layout
        # and storing a reference in the self.widgets dict
        for name in widget_names:
            item = OnOffWidget(name)
            self.controlsLayout.addWidget(item)
            self.widgets.append(item)

        spacer = QSpacerItem(1, 1, QSizePolicy.Policy.Minimum, QSizePolicy.Policy.Expanding)
        self.controlsLayout.addItem(spacer)
        self.controls.setLayout(self.controlsLayout)

        # Scroll Area Properties.
        self.scroll = QScrollArea()
        self.scroll.setVerticalScrollBarPolicy(Qt.ScrollBarPolicy.ScrollBarAlwaysOn)
        self.scroll.setHorizontalScrollBarPolicy(Qt.ScrollBarPolicy.ScrollBarAlwaysOff)
        self.scroll.setWidgetResizable(True)
        self.scroll.setWidget(self.controls)

        # Search bar.
        self.searchbar = QLineEdit()
        self.searchbar.textChanged.connect(self.update_display)

        # Adding Completer.
        self.completer = QCompleter(widget_names)
        self.completer.setCaseSensitivity(Qt.CaseSensitivity.CaseInsensitive)
        self.searchbar.setCompleter(self.completer)

        # Add the items to VBoxLayout (applied to container widget)
        # which encompasses the whole window.
        container = QWidget()
        containerLayout = QVBoxLayout()
        containerLayout.addWidget(self.searchbar)
        containerLayout.addWidget(self.scroll)

        container.setLayout(containerLayout)
        self.setCentralWidget(container)

        self.setGeometry(600, 100, 800, 600)
        self.setWindowTitle('Control Panel')

    def update_display(self, text):

        for widget in self.widgets:
            if text.lower() in widget.name.lower():
                widget.show()
            else:
                widget.hide()


app = QApplication(sys.argv)
w = MainWindow()
w.show()
sys.exit(app.exec())


## 22.3 QCompleter in combobox

In below example, we use a QComoBox to replace the QLineEdit. The completer.setFilterMode(Qt.MatchFlag.MatchContains) change the default filter which allows you to match the text differently

In [ ]:
import sys
from PyQt6.QtWidgets import (QWidget, QLabel, QApplication, QVBoxLayout, QCompleter, QComboBox)
from PyQt6.QtCore import Qt


class MainWindow(QWidget):

    def __init__(self, *args, **kwargs):
        super().__init__()

        self.setGeometry(100, 100, 400, 300)
        self.setWindowTitle('Control Panel')

        # create a main layout
        layout = QVBoxLayout()
        self.setLayout(layout)

        # create a completer based on a list
        names = [
            "Heater", "Stove", "Living Room Light", "Balcony Light",
            "Fan", "Room Light", "Oven", "Desk Light",
            "Bedroom Heater", "Wall Switch"
        ]
        self.completer = QCompleter(names)
        # set the completer as case-insensitive
        self.completer.setCaseSensitivity(Qt.CaseSensitivity.CaseInsensitive)
        self.completer.setFilterMode(Qt.MatchFlag.MatchContains)
        # This enum specifies how completions are provided to the user.
        # - QCompleter.PopupCompletion: Current completions are displayed in a popup window.
        #
        # - QCompleter.InlineCompletion: Completions appear inline (as selected text).
        #
        # - QCompleter.UnfilteredPopupCompletion: All possible completions are displayed in a popup window with the
        #   most likely suggestion indicated as current.
        # self.completer.setCompletionMode(QCompleter.PopupCompletion)

        # create a combobox as Search bar.
        self.searchbar = QComboBox()
        # make it editable
        self.searchbar.setEditable(True)

        # add a action
        self.searchbar.activated.connect(self.update_display)
        # populate the combobox
        for name in names:
            self.searchbar.addItem(name)

        # Add Completer to search bar
        self.searchbar.setCompleter(self.completer)

        self.messageView = QLabel()

        layout.addWidget(self.searchbar)
        layout.addWidget(self.messageView)
        self.show()

    def update_display(self):
        self.messageView.setText(self.searchbar.currentText())


app = QApplication(sys.argv)
w = MainWindow()
w.show()
sys.exit(app.exec())
